In [56]:
# Construct a dictionary for replacing emojis into words
emoji_dict = dict()
with open("pos_emoji.txt", "r") as f:
    for line in f:
        emoji_dict[line.strip()] = 'Positive'

with open("extrpos_emoji.txt", "r") as f:
    for line in f:
        emoji_dict[line.strip()] = 'Positive Positive'

with open("neg_emoji.txt", "r") as f:
    for line in f:
        emoji_dict[line.strip()] = 'Negative'
        
with open("extrneg_emoji.txt", "r") as f:
    for line in f:
        emoji_dict[line.strip()] = 'Negative Negative'

In [52]:
# Construct a set with negative words
negWords = set(open('negative-words.txt').read().split())
# Construct a set with positive words
posWords = set(open('positive-words.txt').read().split())

In [55]:
from collections import OrderedDict
from itertools import groupby, chain, islice
import re
import csv

### islice(g, 2) e.g. booosst -> boosst, juuuuuust -> juust
### TODO: after filter once, still need some operations to make juust, damnn, pleaasee -> just, damn, please
def replaceRepeatedChar(sentence):
    return ''.join(chain.from_iterable(islice(g, 2) for s, g in groupby(sentence.lower())))

def replaceURL(sentence):
    return re.sub(r"http\S+", "|U|", sentence)

### find all tag name (won't be used, just sub function for replaceNameTag)
def findTagName(sentence):
    return re.compile(r'\B@\w+').findall(sentence) 

def replaceNameTag(sentence):
    return re.sub(r'\B@\w+\S+', "|T|", sentence)

def replaceEmotion(sentence):    
    words = sentence.split()
    result = []
    for word in words:
        for key in emoji_dict.iterkeys():
            if key in word:
                result.append(emoji_dict.get(key))
    pos_count, neg_count = 0, 0
    for x in result:
        if x == 'Positive':
            pos_count +=1
        elif x == 'Positive Positive':
            pos_count +=3
        elif x == 'Negative':
            neg_count +=1
        else:
            neg_count +=3
    return pos_count,neg_count

def replaceNegations(sentence):
    words = sentence.split()
    count = 0
    for word in words:
        if word in negWords:
            print word
            count += 1
    return count

def replacePositive(sentence):
    words = sentence.split()
    count = 0
    for word in words:
        if word in posWords:
            print word
            count += 1
    return count

def countPosNeg(sentence):
    words = sentence.split()
    pos_count, neg_count = 0, 0
    for w in words:
        if w in posWords:
            pos_count+=1
        if w in negWords:
            neg_count+=1
    return pos_count,neg_count

def removeStopwords(sentence):
    for word in stopwords[0]:
        regex = re.compile(r'\b('+word+r')\b')
        sentence = regex.sub("-", sentence.lower())  
    return sentence.strip()

In [ ]:
def feature(sentence):
    emoji_pos, emoji_neg = replaceEmotion(sentence)
    word_pos, word_neg = countPosNeg(sentence)
    feat = [1]               # bias term
    feat.append(emoji_pos)   # scores of positive emoji
    feat.append(emoji_neg)   # scores of negative emoji
    feat.append(word_pos)    # positive word counts
    feat.append(word_neg)    # negative word counts
    return feat

In [59]:
ss="iejo qwi XD jej dqjdi :( dide :-| dhewuhe :\ gtd :'( s LOL"

sss_p, sss_n= replaceEmotion(ss)
sss_p, sss_n

(6, 5)

In [16]:
# Construct an set of stopwords
stopwords = []
with open("stop-word-list.txt",'r') as f:
    for word in f:
        stopwords.append(word)
stopwords = set()

In [1]:
'''
### Test function ###
s1 = "I likeeeee move ittt movee it"
print 'Test replaceRepeatedChar function:'
print s1
print replaceRepeatedChar(s1), '\n'

# second time filter (not sure how to do here, maybe retrieve some degree of frequency and replace them)
print 'Filter those meaningless words:'
## assume after added into dict, some example may fail such as pleaasee -> pleas
meaningless_test = ['likee', 'itt', 'movee', 'juust', 'fuuck', 'pleaasee', 'biigg', 'aahh']
print meaningless_test
print map(lambda x: "".join(OrderedDict.fromkeys(x)), meaningless_test), '\n'

# Replace URL
s2 = 'aaahhh... The taste of Italy http://www.caffevita.com/shop/images/BialettiOldSchool.jpg'
print 'Test replaceURL function:'
print s2
print replaceURL(s2), '\n'

# Replace tag Name
tagName = "  -@otto where are you @@@chelin you are so smart! &it @chienhan let's paly pokemongo"
print 'Test findTagName function(not important):'
print tagName
print findTagName(tagName), '\n'
print 'Test replaceNameTag function:'
print replaceNameTag(tagName), '\n'

# Replace emotion
emojiText = 'I got 100 Pikachus with hats and it is so amazing :3'
print 'Test replaceEmotion function:'
print emojiText
print replaceEmotion(emojiText), '\n'

# Replace negative words
negativeText = 'I type in fumble snags coward horrendous these words'
print 'Test replaceNegations function:'
print negativeText
print replaceNegations(negativeText),'\n'

# Replace positive words
positiveText = 'I can spell accurately beauty glitz!'
print 'Test replacePositive function:'
print positiveText
print replacePositive(positiveText),'\n'

# Remove stopwords
stopword_test = 'no no no I my me we our us Hello Pika!!!'
print 'Test removeStopwords function:'
print stopword_test
print removeStopwords(stopword_test)
### End of Testing ###
'''

'\n### Test function ###\ns1 = "I likeeeee move ittt movee it"\nprint \'Test replaceRepeatedChar function:\'\nprint s1\nprint replaceRepeatedChar(s1), \'\n\'\n\n# second time filter (not sure how to do here, maybe retrieve some degree of frequency and replace them)\nprint \'Filter those meaningless words:\'\n## assume after added into dict, some example may fail such as pleaasee -> pleas\nmeaningless_test = [\'likee\', \'itt\', \'movee\', \'juust\', \'fuuck\', \'pleaasee\', \'biigg\', \'aahh\']\nprint meaningless_test\nprint map(lambda x: "".join(OrderedDict.fromkeys(x)), meaningless_test), \'\n\'\n\n# Replace URL\ns2 = \'aaahhh... The taste of Italy http://www.caffevita.com/shop/images/BialettiOldSchool.jpg\'\nprint \'Test replaceURL function:\'\nprint s2\nprint replaceURL(s2), \'\n\'\n\n# Replace tag Name\ntagName = "  -@otto where are you @@@chelin you are so smart! &it @chienhan let\'s paly pokemongo"\nprint \'Test findTagName function(not important):\'\nprint tagName\nprint findTa

In [14]:
import csv
import string

punctuation = set(string.punctuation)
data_text = []
data_sentiment = []
count = 0
emoji = 0
data_emoji = []
### modify the data processing here

with open('Sentiment Analysis Dataset.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        ## Skip header
        if row[1] == "Sentiment":
            continue
        data_sentiment.append(float(row[1]))
        if count <= 100000:
            if replaceEmotion(row[3]) >= 1:
                emoji += 1
        else:
            break;
        count += 1
        ### Ignore capitalization and remove punctuation
        r = ''.join([c for c in row[3].lower() if not c in punctuation])
        data_text.append(r.split())

print "Data size:", len(data_sentiment)
print "Number of Emoji", emoji

:3
T_T
&lt;3
&lt;3
:]
&lt;3
&lt;3
&lt;3
=D
:[
:-D
&lt;3
&lt;3
&lt;3
=D
&lt;3
&lt;3
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
=D
=D
&lt;3
:3
&lt;3
:3
:[
&lt;3
&lt;3
:3
:]
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:[
:]
&lt;3
&lt;3
&lt;3
:-D
&lt;3
&lt;3
&lt;3
=D
C:
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:]
&lt;3
:3
&lt;3
&lt;3
&lt;3
DX
:-D
&lt;3
:3
&lt;3
&lt;3
&lt;3
=D
&lt;3
:[
&lt;3
&lt;3
&lt;3
DX
&lt;3
&lt;3
&lt;3
&lt;3
T_T
:3
D;
:3
:]
:c
:3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
D;
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
=D
&lt;3
&lt;3
:3
&lt;3
:]
&lt;3
=D
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
&lt;3
C:
&lt;3
&lt;3
&lt;3
:[
&lt;3
&lt;3
&lt;3
:o)
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
:]
&lt;3
&lt;3
:3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
DX
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
=D
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
&lt;3
D;
:[
&lt;3
&lt;3
:3
&lt;

In [ ]:
# check first data point
print data_text[0]
print data_sentiment[0]

In [ ]:
import numpy as np
import scipy.optimize
from collections import defaultdict
from sklearn import linear_model

### Producing feature using word count

wordCount = defaultdict(int)

for x in data_text:
    for y in x:
        wordCount[y] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
print "Number of unique words" , len(counts)
print ""
print "Popular words", counts[:10]        

words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [ ]:
### modify here to produce desire feature

def feature(datum):
    feat = [0]*len(words)
    for w in datum:
        if w in wordSet:
            feat[wordId[w]] += 1.0
    feat.append(1.0) #offset
    return feat

### Sentiment analysis

X = [feature(d) for d in data_text]
y = data_sentiment
K = 50000 ### numbers of training data

#No regularization
#theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

#With regularization
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X[:K], y[:K])
theta = clf.coef_

P = 100000### numbers of prediction data
### here using all data to test accuracy
predictions = clf.predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "\nAccuracy", accuracy

In [7]:

### Logistic regression example 
logisticclf = linear_model.LogisticRegression()
logisticclf.fit(X[:K], y[:K])
predictions = logisticclf.predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "Accuracy", accuracy

Accuracy 0.75398


In [8]:

### Naive Bayes example
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
predictions = gnb.fit(X[:K], y[:K]).predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "Accuracy", accuracy

Accuracy 0.69288


In [ ]:

### SVM example warning: take long time to train, Don't try
'''
from sklearn import svm
svmclf = svm.SVC()
svmclf.fit(X[:K], y[:K]) 
predictions = svmclf.predict.predict(X[:P])
accuracy = float(reduce(lambda x,y:x+y,map(lambda x: 1 if round(x[0]) == x[1] else 0, zip(predictions,y[:P]))))/P
print "Accuracy", accuracy